In [8]:
!pip install numpy
!pip install xlrd

In [9]:
import numpy as np
import xlrd as xlrd
import matplotlib.pyplot as plt

In [156]:
# 系统编码规则
# 系统枚举
systemEnums = {
    "总厂":["烧成系统","粉墨系统"],
    "原料预处理":["钙质原料预处理","硅质原料预处理","铝质原料预处理","铁质原料预处理","自定义系统"],
    "原料混合":["钙质原料预均化库","硅质原料预均化库","铝质原料预均化库","铁质原料预均化库","原料预粉磨系统","原料粉磨系统","原料烘干系统","生料输送系统","生料均化系统","自定义系统"],
    "熟料烧成":["生料均化库系统","生料入窑喂料系统","烧成窑尾系统","废气处理系统","旁路放风系统","烧成窑中系统","烧成窑头系统","熟料收尘系统","熟料输送系统","自定义系统"],
    "水泥制备":["熟料储存库","熟料输送系统","混和材破碎系统","混和材烘干","水泥预粉磨系统","水泥粉系统","水泥输送系统","矿渣粉磨系统","自定义系统"],
    "水泥包装运输":["水泥储存库","水泥出库系统","水泥包装","水泥成品发运","水泥汽车散装","熟料汽车散装","水泥火车散装","熟料火车散装","水泥大袋包装","自定义系统"],
    "燃料制备":["原煤破碎系统","煤预均化堆场系统","煤粉制备系统","煤粉收集系统","煤粉输送管道系统","储油系统","燃油系统","XX气体储存及输送系统","自定义系统"],
    "辅助生产设备":["中央化验室","综合材料库","耐火材料库","压缩空气站","压缩空气管道","工艺管道及设备外保温","自定义系统"]
}
# 根据系统编码,获取对应的系统名称
def mapSystemEnums(name):
    systemName = "未知系统代码"
    if name and name != "'" and name != "?":
        analysis = name.split('-')
        if len(analysis) > 1 and analysis[0].isdigit() and analysis[1].isdigit():

            systemIndex = int(analysis[0])
            childSystemIndex = int(analysis[1])
            enumKeys = list(systemEnums.keys())

            if len(enumKeys) > systemIndex:
                system = enumKeys[systemIndex]
                systemChildName = ''

                if childSystemIndex > 0 and len(systemEnums[system]) >= childSystemIndex:
                    systemChildName = systemEnums[system][childSystemIndex - 1]
                    systemName = system + "-" + systemEnums[system][childSystemIndex - 1]
                else:
                    systemName = system + "- 未知子系统代码" 
        elif len(analysis) == 1 and analysis[0].isdigit():
            systemIndex = int(analysis[0])
            enumKeys = list(systemEnums.keys())
            if len(enumKeys) > systemIndex:
                systemName = enumKeys[systemIndex]  + "- 未知子系统代码" 

    return systemName

In [157]:
# 遍历所有行，找出所有不重复项
def divideGroup(data):
    # res = { name: {total, index = []}}
    res = {}
    for index, elm in enumerate(data):
        if elm in res:
            res[elm]['total'] = res[elm]['total'] + 1
            res[elm]['index'].append(index + 2)
        else:
            res[elm] = { 'total' : 1, 'index' : [index + 2] }     
    print(res)
    for key in res.keys():
        #print(key + " " + str(res[key]['total']) + " " + str(res[key]['index']))
        print("编号： " + key + "   数量： " + str(res[key]['total']))
        #print(str(key) + "  " + str(res[key]['index']) + "\n")

# 对单一维度进行分析
def analysisDimension(sheet, colIndex, eIndex,rowRange):
    colData = sheet.col_values(colIndex)
    colData_cut = colData[rowRange[0]:rowRange[1]]
    divideGroup(colData_cut)

In [273]:
# 对多个维度进行分析
def analysisDimensions(sheet, rowRange, systemName, sIndex, eIndex):

    tables = []

    rows = sheet.get_rows()
    for index, row in enumerate(rows):
        if index >= (rowRange[0]-1) and index < rowRange[1] :
            newRow = []
            for j, col in enumerate(row):
                strCol = str(col)[6:]
                newCol = strCol[:-1]
                newRow.append(newCol)
                
            newRow.append(index+1)
            tables.append(newRow)

        elif index > rowRange[1]:
            break
#     print('tables',tables)
    
    tree = {}
    for i,row in enumerate(tables):
        name = row[sIndex]
        child = formatData(tables,name,sIndex,eIndex)
        child["name"] = mapSystemEnums(name)
        tree[name] = child
#     print('tree：',tree)
#     return
#     if systemName:
#         singlePrintTree(tree,systemName)
#     else:
    allPrintTree(tree)
# 递归处理的分级
def handleKeys(handleList,name,contrastList,index,rowNumList,labelName):
    nameSplit = handleList
    saveName = nameSplit[0]
    if len(nameSplit) > 1:
        del nameSplit[0]
    if len(nameSplit) > 1:
        if not(nameSplit[0] in contrastList):
            print(index * "      " + "└── " + nameSplit[0])
        contrastList.append(nameSplit[0])
        analysisSortBranch(nameSplit,saveName,contrastList,index+1,rowNumList,labelName)  
    elif name != saveName:
        rows = ""
#         print(rowNumList,nameSplit)
        for row in rowNumList[labelName]:
            rowLastIndex = list(row.keys())[-1]
            if rows:
                rows += ","+str(row[rowLastIndex])
            else:
                rows = str(row[rowLastIndex])
        print(index * "      " + "└──── " + nameSplit[-1] + " 【行号:" + " "+ rows +"】")

# 递归解析多级设备
def analysisSortBranch(handleList,name,contrastList,index,rowNumList,labelName): 
    if index == 1:
        for i,keyName in enumerate(handleList):
            handleKeys(keyName.split('-'),name,contrastList,index,rowNumList,keyName)
    else:
        handleKeys(handleList,name,contrastList,index,rowNumList,labelName)
# 遍历传入的数组,根据系统(下标:2)和设备名称(下标:3)第一个分隔符"-"前的设备名称,两级分类绘制树结构
def formatData(list,name,si,ei):
    branch = {}
    for i,row in enumerate(list):
        if row[si] == name:
            # 截取设备一级名称
            equipment = row[ei].split('-')[0]
            
            # 将行数据(list类型)转为对象；对象最后一个属性为当前行在Excel的行数

            child = {}
            column = 0
            for j,col in enumerate(row):
                if isinstance(col,int):
                    column = col
                child[str(j)] = col
                
            if equipment in branch:
#                 branch[equipment].append(obj)
                if row[ei] in branch[equipment]:
                    branch[equipment][row[ei]].append(child)
                else:
                    branch[equipment][row[ei]] = [child]
                
            else:
                branch[equipment] = {
                    row[ei]: [child]
                }

    return branch

# 展开显示所有
def allPrintTree(tree):
    systemList = tree.keys()
    for s in systemList:
        print(s + "【"+ tree[s]["name"] +"】")
        equipmentList = list(tree[s].keys())
        for child in equipmentList:
            if child != "name":
                branchList = tree[s][child]
                equipment = list(branchList.keys())
                sortList = sorted(branchList.keys())
                testKeys = list(sortList)
                if len(testKeys) > 1:
                    print(1 * "   " + "└── " + child)
#                     newObj = {}
#                     for eName in testKeys:
#                         objKey = eName.split('-')[-1]
#                         newObj[objKey] = branchList[eName]
#                         newObj[eName] = branchList[eName]

#                     print('branchList',branchList)
#                     return
                    analysisSortBranch(testKeys,"",[],1,branchList,"")
                else:
                    e = equipment[0]
                    columns = ""
                    for col in tree[s][child][e]:
                        lastIndex = list(col.keys())[-1]
                        if columns:
                            columns += ","+str(col[lastIndex])
                        else:
                            columns = str(col[lastIndex])
                    print(1 * "   " + "└── " + child + " 【行号:" + " "+ columns +"】")
                
# 单个查找
def singlePrintTree(tree,systemName):
    if systemName in tree:
        fChilds = tree[systemName]
        print(systemName + "【"+ fChilds["name"] +"】")
        fList = fChilds.keys()
        for s in fList:
            if s != "name":
                fChild = list(fChilds[s].keys())
                print(1 * "   " + "└── " + s)
                for e in fChild:
                    columns = ""
                    for col in fChilds[s][e]:
                        lastIndex = list(col.keys())[-1]
                        if columns:
                            columns += ","+str(col[lastIndex])
                        else:
                            columns = str(col[lastIndex])
                    
                    print(3 * "   " + "└──── " + e + " 【行号:" + " "+ columns +"】")
    else:
        print('未找到系统',systemName)
          

In [274]:
# todo执行设备和系统维度的分析

# inputParams:
#   sheet: excel的表格对象; type = Sheet Object
#   osCol: 系统维度的列索引; type = int
#   eqCol: 设备维度的列索引; type = int
#   rowRange: 要分析的行区间; type = List[]
#def analysisEqAndSystem(sheet, osCol, eqCol, rowRange):
#     osData = sheet.col_values(osCol)[rowRange[0]:rowRange[1]]
#     eqData = sheet.col_values(eqCol)[rowRange[0]:rowRange[1]]
#     os_eq_Data = []
#     for index, eqName in enumerate(eqData):
#         os_eq = {}
#         os_eq['os'] = osData[index]
#         os_eq['eq'] = eqName.split('-')
#         os_eq_Data.append(os_eq)
    
#     #print(os_eq_Data)
    
#     # os tree
#     os_eq_tree = {}
#     for osName in osData:
#         osNameSplit = osName.split('-')
#         father_os_name = osNameSplit[0]
#         child_os_name = osNameSplit[1]
#         if(father_os_name in os_eq_tree):
#             father_node = os_eq_tree[father_os_name]
#             print(child_os_name + " " + str(father_node.isIncludeNode(child_os_name)))
#             if(father_node.isIncludeNode(child_os_name) == False):
#                 child_node = Node(child_os_name)
#                 father_node.addSonNode(child_node)
#         else:
#             father_node = Node(father_os_name)
#             os_eq_tree[father_os_name] = father_node
            
#     return os_eq_tree

In [275]:
# 树结构的Node类
class Node:
    def __init__(self, name, index):
        self.name = str(name)
        self.son = []
        self.index = index
    
    # inputParams: Class Node
    def addSonNode(self, *nodes):
        for node in nodes:
            self.son.append(node)
    
    # inputParams: String node_name
    def isIncludeNode(self, node_name):
        if(len(self.son) == 0):
            return False
        else:
            for node in self.son:
                if(node_name == node.name):
                    return True
                else:
                    return False
            
    # inputParams: num: pre space number
    def printSon(self, depth):
        if(len(self.son) != 0):
            for son in self.son:
                print(depth * "   " + "└──" + son.name + " " + str(son.index))
                son.printSon(depth+2)

### Test: Node

In [276]:
# root = Node("root", 2)

# son1 = Node("son1", 3)
# son2 = Node("son2", 3)

# son1_1 = Node("son1_1", 4)
# son1_2 = Node("son1_2", 6)
# son1_3 = Node("son1_3", 3)

# son1_1_1 = Node("son1_1_1", 3)
# son1_1_2 = Node("son1_1_2", 5)

# son2_1 = Node("son2_1", 6)
# son2_2 = Node("son2_2", 6)

# root.addSonNode(son1,son2)
# son1.addSonNode(son1_1,son1_2,son1_3)
# son1_1.addSonNode(son1_1_1,son1_1_2)

# son2.addSonNode(son2_1,son2_2)

# print(root.name)
# son1.printSon(1)

In [287]:
def funcHandle(arr,before):
    newRow = arr
    node = Node(newRow[0],1)
    before.addSonNode(node)
    del newRow[0]
    if len(newRow) > 0:
        result(newRow,node)

def result(arr,before):
    if before.name == "root":
        for row in arr:
            newRow = row.split('-')
            funcHandle(newRow,before)
    else:
        funcHandle(arr,before)

data = [
    "窑-辅传电机",
    "窑-辅传电机",
    "风冷套风机1",
    "风冷套风机2",
    "均化库-罗茨风机",
    "配电柜总进线",
    "空压机-控制箱1"
]

rootName = Node("root",2)
result(data,rootName)
print(rootName.name)
rootName.printSon(1)

root
   └──窑 1
         └──辅传电机 1
   └──窑 1
         └──辅传电机 1
   └──风冷套风机1 1
   └──风冷套风机2 1
   └──均化库 1
         └──罗茨风机 1
   └──配电柜总进线 1
   └──空压机 1
         └──控制箱1 1


## Main Function

In [280]:
# 导入Excl
workbook = xlrd.open_workbook("/Users/cmx/Documents/tags初步分类处理.xls")
# workbook = xlrd.open_workbook("/Users/cmx/Documents/cmxtag.xls")

sheetName = workbook.sheet_names()[0]
sheet = workbook.sheet_by_name(sheetName)

In [285]:
# 行号范围;
# rowRange = [375, 376]
rowRange = [2, 2490]
# rowRange = [2, 24]





# 系统维度列号
osCol = 2

# 设备维度列号
eqCol = 3

# 需要查找的系统名称
# systemName = '3-4'
systemName = ''
# t实现设备和系统维度的分析
# analysisDimension(sheet, osCol, eqCol, rowRange)
analysisDimensions(sheet, rowRange, systemName, osCol, eqCol)


# 

3-4【熟料烧成-废气处理系统】
   └── 回灰斗提 【行号: 2】
   └── 窑尾
      └──── 窑尾 【行号: 1594,1608,2354,2355,2356】
      └──── 在线烟气分析仪 【行号: 265,266,267,268,269,270,1596】
      └── 大收尘器
            └──── 控制柜 【行号: 390,391,392】
      └── 废气风机
            └──── 主电机 【行号: 8,614,615,616】
            └──── 主电机冷却器 【行号: 387,388,389】
            └──── 主电机加热器 【行号: 617】
            └──── 主电机变频器柜 【行号: 384,385,386】
      └── 排风机
            └──── 主电机 【行号: 12,13,14】
            └──── 出口 【行号: 6,7】
            └── 变频柜
                  └──── 电机 【行号: 10,11】
            └──── 进口 【行号: 9】
            └── 风机
                  └──── 轴承 【行号: 15,16】
      └── 除尘1#
            └──── 拉链机 【行号: 403】
      └── 除尘2#
            └──── 拉链机 【行号: 404】
      └──── 除尘下料器 【行号: 393,394,395,396,397,398,399,400,401,402】
   └── 循环风机
      └──── 前轴承 【行号: 17,18】
      └──── 后轴承 【行号: 19,20】
      └──── 阀门 【行号: 374】
   └── 袋收尘机 【行号: 375】
   └── 袋收尘器 【行号: 376】
   └── 回灰链式输送机 【行号: 405,406,407,408,409,410】
   └── #3回灰斗式提升机
      └──── #3回灰斗式提升机 【行号: 411,41